<a href="https://colab.research.google.com/github/ezevischi/capstone/blob/master/Capstone_Project_The_Battle_of_Neighborhoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In the city where I live, I would like to know where it would be more convenient on a regular basis to fill my car with gasoline. Therefore I thought of two options, fill it near my home or near work, in which I go by car. With Foursquare I can do the analysis and decide in which neighborhood I have more options. In turn, he would need to fill up gasoline at "Shell" brand service stations. Let's see what information I get by comparing two neighborhoods in the city of Buenos Aires

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Folium installed
Libraries imported.


In [2]:
CLIENT_ID = 'MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2' # your Foursquare ID
CLIENT_SECRET = 'YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 40
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2
CLIENT_SECRET:YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI


In [3]:
#address of my office
address = 'Av. del Libertador 7202, Buenos Aires'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-34.5467627 -58.4576083


In [11]:
#I want to know the fuel stations near to my office
search_query = 'Shell'
radius = 1500
print(search_query + ' .... OK!')

Shell .... OK!


In [12]:
#I look at in Foursquare
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2&client_secret=YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI&ll=-34.5467627,-58.4576083&v=20180604&query=Shell&radius=1500&limit=40'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f459a9fcd347610a72628a7'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'primary': True,
      'shortName': 'Gas Station'}],
    'hasPerk': False,
    'id': '4c7321eed944224b0dbb0e29',
    'location': {'address': 'Av. Figueroa Alcorta 7600',
     'cc': 'AR',
     'city': 'Buenos Aires',
     'country': 'Argentina',
     'crossStreet': 'esquina Udaondo',
     'distance': 529,
     'formattedAddress': ['Av. Figueroa Alcorta 7600 (esquina Udaondo)',
      'Buenos Aires',
      'Buenos Aires C.F.',
      'Argentina'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -34.54653639219639,
       'lng': -58.451841698590336}],
     'lat': -34.54653639219639,
     'lng': -58.451841698590336,
     'state': 'Buenos Aires C.F.'},
    'name': 'Shell'

In [14]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode,venuePage.id
0,4c7321eed944224b0dbb0e29,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397256,False,Av. Figueroa Alcorta 7600,esquina Udaondo,-34.546536,-58.451842,"[{'label': 'display', 'lat': -34.5465363921963...",529,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Figueroa Alcorta 7600 (esquina Udaondo), ...",NaN,NaN
1,4bfc2a47d6f2c9b610ce4ec8,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397256,False,Av. del Libertador 7920,esq. Av. Comodoro Martín Rivadavia,-34.541944,-58.463151,"[{'label': 'display', 'lat': -34.5419444839943...",738,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. del Libertador 7920 (esq. Av. Comodoro Ma...,C1429BMZ,NaN
2,5f35708c83a8781d8ca91e6f,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397256,False,Avenida Presidente Figueroa Alcorta 7786,NaN,-34.546543,-58.451910,"[{'label': 'display', 'lat': -34.5465429, 'lng...",523,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 7786, 142...",1429,594433018
3,5f419429aee48d48c767649e,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397256,False,AV.DEL LIBERTADOR 7920-90,NaN,-34.541921,-58.463166,"[{'label': 'display', 'lat': -34.541921, 'lng'...",741,AR,CAPITAL FEDERAL,Buenos Aires,Argentina,"[AV.DEL LIBERTADOR 7920-90, 1428 CAPITAL FEDER...",1428,590458264
4,4bf4aa14e5eba59363fe1f90,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397256,False,Avenida Cabildo 4701,Deheza,-34.541197,-58.473732,"[{'label': 'display', 'lat': -34.54119731, 'ln...",1602,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Avenida Cabildo 4701 (Deheza), 1429 Buenos Ai...",1429,NaN


In [15]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Shell,Gas Station,Av. Figueroa Alcorta 7600,esquina Udaondo,-34.546536,-58.451842,"[{'label': 'display', 'lat': -34.5465363921963...",529,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Figueroa Alcorta 7600 (esquina Udaondo), ...",NaN,4c7321eed944224b0dbb0e29
1,Shell,Gas Station,Av. del Libertador 7920,esq. Av. Comodoro Martín Rivadavia,-34.541944,-58.463151,"[{'label': 'display', 'lat': -34.5419444839943...",738,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. del Libertador 7920 (esq. Av. Comodoro Ma...,C1429BMZ,4bfc2a47d6f2c9b610ce4ec8
2,Shell,Gas Station,Avenida Presidente Figueroa Alcorta 7786,NaN,-34.546543,-58.451910,"[{'label': 'display', 'lat': -34.5465429, 'lng...",523,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 7786, 142...",1429,5f35708c83a8781d8ca91e6f
3,Shell,Gas Station,AV.DEL LIBERTADOR 7920-90,NaN,-34.541921,-58.463166,"[{'label': 'display', 'lat': -34.541921, 'lng'...",741,AR,CAPITAL FEDERAL,Buenos Aires,Argentina,"[AV.DEL LIBERTADOR 7920-90, 1428 CAPITAL FEDER...",1428,5f419429aee48d48c767649e
4,Shell,Gas Station,Avenida Cabildo 4701,Deheza,-34.541197,-58.473732,"[{'label': 'display', 'lat': -34.54119731, 'ln...",1602,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Avenida Cabildo 4701 (Deheza), 1429 Buenos Ai...",1429,4bf4aa14e5eba59363fe1f90
5,Shell,Gas Station,Cramer,Virrey del Pino,-34.556951,-58.470301,"[{'label': 'display', 'lat': -34.5569506781473...",1624,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Cramer (Virrey del Pino), Buenos Aires, Bueno...",NaN,535fa191498eb57b90d2b7ca
6,Shell departamento de ingeniería,Coworking Space,Av. Cabildo 4441,NaN,-34.551313,-58.468204,"[{'label': 'display', 'lat': -34.5513128931675...",1095,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Cabildo 4441, Buenos Aires, Buenos Aires ...",NaN,513e0bf8e4b06a8710614e56


In [16]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my office

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Office',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [28]:
#address of my house
address = 'Beruti 3039, Buenos Aires'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

-34.58928185714286 -58.40671626938776


In [29]:
#I want to know the fuel stations near to my house
search_query = 'Shell'
radius = 1500
print(search_query + ' .... OK!')

Shell .... OK!


In [30]:
#I look at in Foursquare near my house
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=MBQBAT4QEUESXYG2QMD1BPHRFNTPJA1QPRXWDE5SIZ30IOM2&client_secret=YNQKOKFRAVV3NLMRW3C0ID43TQ2J0CBJEFAWSH1S2J04VRJI&ll=-34.58928185714286,-58.40671626938776&v=20180604&query=Shell&radius=1500&limit=40'

In [31]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f459c9f1883f35ebf081d40'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/gas_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d113951735',
      'name': 'Gas Station',
      'pluralName': 'Gas Stations',
      'primary': True,
      'shortName': 'Gas Station'}],
    'hasPerk': False,
    'id': '4bbea9f898f495216aabd163',
    'location': {'address': 'Av. Pres. Figueroa Alcorta 3099',
     'cc': 'AR',
     'city': 'Buenos Aires',
     'country': 'Argentina',
     'crossStreet': 'esq. Mcal. Ramón Castilla',
     'distance': 1244,
     'formattedAddress': ['Av. Pres. Figueroa Alcorta 3099 (esq. Mcal. Ramón Castilla)',
      'C1425CKJ Buenos Aires',
      'Buenos Aires C.F.',
      'Argentina'],
     'labeledLatLngs': [{'label': 'display',
       'lat': -34.57964470524101,
       'lng': -58.39983577134228}],
     'lat': -34.57964470524101,
     'lng': -58.39983577134228,
     'postalCod

In [32]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id
0,4bbea9f898f495216aabd163,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397613,False,Av. Pres. Figueroa Alcorta 3099,esq. Mcal. Ramón Castilla,-34.579645,-58.399836,"[{'label': 'display', 'lat': -34.5796447052410...",1244,C1425CKJ,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Pres. Figueroa Alcorta 3099 (esq. Mcal. R...,NaN
1,4c680e397abde21e168c6768,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397613,False,Av. Corrientes 3101,esq. Jean Jaures,-34.603932,-58.409133,"[{'label': 'display', 'lat': -34.603932, 'lng'...",1645,C1193AAD,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Corrientes 3101 (esq. Jean Jaures), C1193...",NaN
2,4c7178529c6d6dcbbd85d57a,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397613,False,Av. Córdoba 3700,esq. Jerónimo Salguero,-34.597965,-58.419155,"[{'label': 'display', 'lat': -34.5979646957906...",1494,C1188AAP,AR,Almagro,Buenos Aires C.F.,Argentina,"[Av. Córdoba 3700 (esq. Jerónimo Salguero), C1...",NaN
3,5f3821609e84c4203d4d25f4,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397613,False,SAN LUIS 3187,NaN,-34.599150,-58.412524,"[{'label': 'display', 'lat': -34.59915, 'lng':...",1220,1186,AR,CAPITAL FEDERAL,Buenos Aires,Argentina,"[SAN LUIS 3187, 1186 CAPITAL FEDERAL, Buenos A...",591689397
4,5f3d5841e620b23fe1be41d6,Shell,"[{'id': '4bf58dd8d48988d113951735', 'name': 'G...",v-1598397613,False,Avenida Presidente Figueroa Alcorta 3099,NaN,-34.579619,-58.399836,"[{'label': 'display', 'lat': -34.579619, 'lng'...",1246,1425,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 3099, 142...",590449912


In [33]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Shell,Gas Station,Av. Pres. Figueroa Alcorta 3099,esq. Mcal. Ramón Castilla,-34.579645,-58.399836,"[{'label': 'display', 'lat': -34.5796447052410...",1244,C1425CKJ,AR,Buenos Aires,Buenos Aires C.F.,Argentina,[Av. Pres. Figueroa Alcorta 3099 (esq. Mcal. R...,4bbea9f898f495216aabd163
1,Shell,Gas Station,Av. Corrientes 3101,esq. Jean Jaures,-34.603932,-58.409133,"[{'label': 'display', 'lat': -34.603932, 'lng'...",1645,C1193AAD,AR,Buenos Aires,Buenos Aires C.F.,Argentina,"[Av. Corrientes 3101 (esq. Jean Jaures), C1193...",4c680e397abde21e168c6768
2,Shell,Gas Station,Av. Córdoba 3700,esq. Jerónimo Salguero,-34.597965,-58.419155,"[{'label': 'display', 'lat': -34.5979646957906...",1494,C1188AAP,AR,Almagro,Buenos Aires C.F.,Argentina,"[Av. Córdoba 3700 (esq. Jerónimo Salguero), C1...",4c7178529c6d6dcbbd85d57a
3,Shell,Gas Station,SAN LUIS 3187,NaN,-34.599150,-58.412524,"[{'label': 'display', 'lat': -34.59915, 'lng':...",1220,1186,AR,CAPITAL FEDERAL,Buenos Aires,Argentina,"[SAN LUIS 3187, 1186 CAPITAL FEDERAL, Buenos A...",5f3821609e84c4203d4d25f4
4,Shell,Gas Station,Avenida Presidente Figueroa Alcorta 3099,NaN,-34.579619,-58.399836,"[{'label': 'display', 'lat': -34.579619, 'lng'...",1246,1425,AR,Capital Federal,Buenos Aires,Argentina,"[Avenida Presidente Figueroa Alcorta 3099, 142...",5f3d5841e620b23fe1be41d6
5,Shell,Gas Station,Avenida Córdoba 3700,NaN,-34.597972,-58.419247,"[{'label': 'display', 'lat': -34.597972, 'lng'...",1501,C1188,AR,AAP,Buenos Aires,Argentina,"[Avenida Córdoba 3700, C1188 AAP, Buenos Aires...",5f4172e1495ff35a98772992


In [34]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around my house

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='House',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

Analyzing the two alternatives. It would be convenient for me to fill my car with gasoline when I arrive or when I leave the office since there are more options available nearby